# Exploring Data From Hacker News

*What is the best time to create a post with the higher chance of receiving comments*


Hacker News is a popular technology site that has 2 modalities of posts:
- Users submit Ask HN posts to ask the Hacker News community a specific question.
- Users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

The aim of this study is to compare the number of comments on both modalities to find out which one receives more comments on average and then discover what is the best time to create a post with a higher chance of receiving comments.

The data set is available here: https://www.kaggle.com/hacker-news/hacker-news-posts

Originally it was 300,000 rows that was reduced to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. 
 
*STEP 1:
open and read the file as a list of lists* 

In [10]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file) #read the file as a list of lists
hn_headers = hn[0] #separete head
hn = hn[1:]

print(hn_headers)
print('\n')

for row in hn[:5]: #test
    print(row)
    print('\n')




['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




*STEP 2: Since I´m only concerned with post titles beginning with "Ask HN" or "Show HN", I'll create new lists of lists containing just the data for those titles with the string method startswith and by controling capitalization with the lower method:*


In [20]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn') == True:
        ask_posts.append(row)
    elif title.startswith('show hn') == True:
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts), len(show_posts), len(other_posts)) #verification

1744 1162 17194


*STEP 3: determine if ask posts or show posts receive more comments on average.*

In [22]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = row[4]
    total_ask_comments += int(num_comments)
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

total_show_comments = 0
for row in show_posts:
    num_comments = row[4]
    total_show_comments += int(num_comments)
    
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)



14.038417431192661
10.31669535283993


As expected, ask posts recived more comments on average. This probably happens because an ask post requiers answers, while show posts can just be observed. Since ask posts are more responsive, I´ll focus the remaining analysis just on them.

*STEP 4: determine if ask posts created at a certain time are more likely to attract comments.* 

4.1. *Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.*

In [27]:
import datetime as dt
counts_by_hour = {}  # the number of ask posts created every hour
comments_by_hour = {}  # the hour of creation of the post and its number of comments 

result_list = []
for row in ask_posts:
    created_at = row[6]
    n_comments = row[4]
    list_ask = [created_at, int(n_comments)]
    result_list.append(list_ask)

for row in result_list:
    datetime_hn = row[0]
    datetime_hn = dt.datetime.strptime(datetime_hn, "%m/%d/%Y %H:%M")
    hour = datetime_hn.hour
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
    

4.2. *Calculate the average number of comments ask posts receive by hour created.*

In [30]:
avg_by_hour = []

for hour in comments_by_hour:
    list_avg = [hour, comments_by_hour[hour]/counts_by_hour[hour]]
    avg_by_hour.append(list_avg)
    
avg_by_hour

[[9, 5.5777777777777775],
 [13, 14.741176470588234],
 [10, 13.440677966101696],
 [14, 13.233644859813085],
 [16, 16.796296296296298],
 [23, 7.985294117647059],
 [12, 9.41095890410959],
 [17, 11.46],
 [15, 38.5948275862069],
 [21, 16.009174311926607],
 [20, 21.525],
 [2, 23.810344827586206],
 [18, 13.20183486238532],
 [3, 7.796296296296297],
 [5, 10.08695652173913],
 [19, 10.8],
 [1, 11.383333333333333],
 [22, 6.746478873239437],
 [8, 10.25],
 [4, 7.170212765957447],
 [0, 8.127272727272727],
 [6, 9.022727272727273],
 [7, 7.852941176470588],
 [11, 11.051724137931034]]

Although is there the results I want, this format makes it hard to identify the hours with the highest values. That is why I´ll sort the list of lists and print the five highest values in a format that's easier to read.

To do that I´ll create a list whose first element is the second element of the row, and whose second element is the first element of the row. So then, I can sort the 1th element in a descending order.

In [46]:
swap_avg_by_hour = []

for row in avg_by_hour:
    first = row[1]
    sec = row[0]
    list_swap = [first, sec]
    swap_avg_by_hour.append(list_swap)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print("Top 5 Hours for Ask Posts Comments")
print('\n')
for row in sorted_swap[:5]: #top 5
    average = float(row[0])
    hour = dt.datetime.strptime(str(row[1]),'%H')
    new_hour = hour.strftime('%H:%M')
    print("{}: {:.2f} average comments per post".format(new_hour, average))

Top 5 Hours for Ask Posts Comments


15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


**CONCLUSION**

Analyzing the results, I conclude that at 15 in the afternoon is the best time to create an ask post with a higher chance of receiving comments.